In [ ]:
#THIS FUNCTION IS FOR RUNNING WITH SLURM JOB ARRAY
#(SPLITS UP JOB_ARRAY BELOW INTO EVEN MORE TASKS)
def StartSlurmJobArray(num_jobs,num_slurm_jobs, ISRUN):
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=1
    if ISRUN==False:
        start_job=1;end_job=num_jobs+1
        return start_job,end_job
    total_elements=num_jobs #total num of variables

    job_range = total_elements // num_slurm_jobs  # Base size for each chunk
    remaining = total_elements % num_slurm_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_slurm_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_slurm_jobs - 1: 
            end_job = total_elements 
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_slurm_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    # if sbatch==True:
        
    start_job, end_job = get_job_range(job_id, num_slurm_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    if start_job==0: start_job=1
    if end_job==total_elements: end_job+=1
    return start_job,end_job

# job_id=1
# [start_slurm_job,end_slurm_job,slurm_index_adjust]=StartSlurmJobArray(num_jobs,num_slurm_jobs,ISRUN)
# parcel=parcel1.isel(xh=slice(start_job,end_job))

In [ ]:
#LOADING LIBRARIES
import os
os.environ["DASK_WORKER_MEMORY_LIMIT"] = "25GB"  #"160GB" for full node

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec

import xarray as xr
import dask  # import after setting env var
import time
import pickle
import h5py

In [ ]:
#MAIN DIRECTORIES
mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
scratchDirectory='/home/air673/koa_scratch/'
codeDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Project_Algorithms/Entrainment'

In [ ]:
#LOADING DATA
def GetDataDirectories(simulationNumber):
    if simulationNumber == 1:
        Directory=os.path.join(mainDirectory,'Model/cm1r20.3/run')
        res='1km'; t_res='5min'; Np_str='1e6'; Nz_str='34'
    elif simulationNumber == 2:
        Directory=scratchDirectory
        res='1km'; t_res='1min'; Np_str='50e6'; Nz_str='95'
    elif simulationNumber == 3:
        Directory=scratchDirectory
        res='250m'; t_res='1min'; Np_str='50e6'; Nz_str='95'
        
    dataDirectory = os.path.join(Directory, f"cm1out_{res}_{t_res}_{Nz_str}nz.nc")
    parcelDirectory = os.path.join(Directory,f"cm1out_pdata_{res}_{t_res}_{Np_str}np.nc")
    return dataDirectory, parcelDirectory, res,t_res,Np_str,Nz_str
    
def GetData(dataDirectory, parcelDirectory):
    dataNC = xr.open_dataset(dataDirectory, decode_timedelta=True) 
    parcelNC = xr.open_dataset(parcelDirectory, decode_timedelta=True) 
    return dataNC,parcelNC

def SubsetDataVars(dataNC):
    varList = ["thflux", "qvflux", "tsk", "cape", 
               "cin", "lcl", "lfc", "th",
               "prs", "rho", "qv", "qc",
               "qr", "qi", "qs","qg", 
               "buoyancy", "uinterp", "vinterp", "winterp",]
    
    varList += ["ptb_hadv", "ptb_vadv", "ptb_hidiff", "ptb_vidiff",
                "ptb_hturb", "ptb_vturb", "ptb_mp", "ptb_rdamp", 
                "ptb_rad", "ptb_div", "ptb_diss",]
    
    varList += ["qvb_hadv", "qvb_vadv", "qvb_hidiff", "qvb_vidiff", 
                "qvb_hturb", "qvb_vturb", "qvb_mp",]
    
    varList += ["wb_hadv", "wb_vadv", "wb_hidiff", "wb_vidiff",
                "wb_hturb", "wb_vturb", "wb_pgrad", "wb_rdamp", "wb_buoy",]

    return dataNC[varList]

[dataDirectory,parcelDirectory, res,t_res,Np_str,Nz_str] = GetDataDirectories(simulationNumber=1)
[data1,parcel1] = GetData(dataDirectory, parcelDirectory)

In [ ]:
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'

In [ ]:
#########################################

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
####################################
# SETUP

In [ ]:
################################
#ENTRAINMENT DATA_TYPE SETUP
PROCESSING=False #if using full entrainment calculations
PROCESSING=True #if using turbulence-filtered entrainment calculations
################################

In [ ]:
################################
#JOB ARRAY SETUP
################################
# how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100
if Np_str=='1e6':
    num_jobs=60 #1M parcels
    num_slurm_jobs=10
elif Np_str=='50e6':
    num_jobs=600 #50M parcels
    num_slurm_jobs=100
##############################

In [ ]:
##############################################
#DATA LOADING FUNCTIONS

In [ ]:
#INITIALIZE DATA FUNCTION
###############################################################
#INITIALIZE DATA FUNCTION
###############################################################
def InitiateArray(out_file,vars,t_chunk_size,z_chunk_size,y_chunk_size,x_chunk_size,t_size=None,z_size=None,y_size=None,x_size=None):
    # Define array dimensions (adjust based on your data)

    if t_size==None:
        t_size = len(data1['time'])  # Number of timesteps
    if z_size==None:
        z_size = len(data1['zh'])    # Number of vertical levels
    if y_size==None:
        y_size = len(data1['yh'])    # Number of vertical levels
    if x_size==None:
        x_size = len(data1['xh'])    # Number of vertical levels
    
    with h5py.File(out_file, 'w') as f: 
        # Check if the dataset 'theta_e' already exists
        for var_name in vars:
            if var_name not in f:
                # Create a dataset with the full size for all time steps (initially empty)
                f.create_dataset(var_name, 
                                 (t_size, z_size, y_size, x_size),  # Full size for all timesteps
                                 chunks=(t_chunk_size, z_chunk_size, y_chunk_size, x_chunk_size))  # Chunks for time axis to allow resizing

def InitiateArray_Job(data,out_file,vars,t_chunk_size,z_chunk_size,y_chunk_size,x_chunk_size,t_size=None,z_size=None,y_size=None,x_size=None):
    # Define array dimensions (adjust based on your data)

    if t_size==None:
        t_size = len(data['time'])  # Number of timesteps
    if z_size==None:
        z_size = len(data1['zh'])    # Number of vertical levels
    if y_size==None:
        y_size = len(data1['yh'])    # Number of vertical levels
    if x_size==None:
        x_size = len(data1['xh'])    # Number of vertical levels
    
    with h5py.File(out_file, 'w') as f: 
        # Check if the dataset 'theta_e' already exists
        for var_name in vars:
            if var_name not in f:
                # Create a dataset with the full size for all time steps (initially empty)
                f.create_dataset(var_name, 
                                 (t_size, z_size, y_size, x_size),  # Full size for all timesteps
                                 chunks=(t_chunk_size, z_chunk_size, y_chunk_size, x_chunk_size))  # Chunks for time axis to allow resizing

In [ ]:
#JOB ARRAY SETUP
def StartJobArray(job_id,num_jobs):
    total_elements=len(data1['time']) #total num of variables

    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()

    # if sbatch==True:
    #     job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    #     if job_id==0: job_id=1
        
    start_job, end_job = get_job_range(job_id, num_jobs)
    index_adjust=start_job
    # print(f'start_job = {start_job}, end_job = {end_job}')
    return start_job,end_job,index_adjust
# job_id=1
# [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)

In [ ]:
# Reading Back Data Later
##############
def make_data_dict(in_file,var_names,read_type,start_job,end_job):
    if read_type=='h5py':
        with h5py.File(in_file, 'r') as f:
            data_dict = {var_name: f[var_name][start_job:end_job] for var_name in var_names}

            
    elif read_type=='xarray':
        in_data = xr.open_dataset(
            in_file,
            engine='h5netcdf',
            phony_dims='sort',
            chunks={'phony_dim_0': 100, 'phony_dim_1': 1_000_000} 
        )
        data_dict = {k: in_data[k][start_job:end_job].compute().data for k in var_names}

    return data_dict

# read_type='xarray'
read_type='h5py'

In [ ]:
def GetData(data1,parcel1,start_job,end_job):
    #Indexing Array with JobArray
    data=data1.isel(time=slice(start_job,end_job))
    parcel=parcel1.isel(time=slice(start_job,end_job))
    return data,parcel
# [data,parcel]=GetData(data1,parcel1,start_job,end_job)

In [ ]:
def GetSpatialData(data,start_job,end_job):
    import h5py
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/OUTPUT/'
    in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'
    
    var_names = ['Z', 'Y', 'X']
    data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
    Z,Y,X = (data_dict[k] for k in var_names)
    
    # #Making Time Matrix
    Nt=len(data['time'])
    T = np.broadcast_to(np.arange(Nt)[:, None], Z.shape)  # shape: (Nt, p) #A_c==>Z
    
    # check_memory(globals())
    return T,Z,Y,X

In [ ]:
def GetAData(PROCESSING,start_job,end_job):
    if PROCESSING==False:
        dir2=dir+'Project_Algorithms/Lagrangian_Arrays/OUTPUT/'
        in_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'
        
        var_names = ['A_g', 'A_c']
        data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
        A_g, A_c = (data_dict[k] for k in var_names) #, W
    
    elif PROCESSING==True:
        dir2=dir+'Project_Algorithms/Entrainment/OUTPUT/'
        in_file=dir2+f'processed_binary_arrays_{res}_{t_res}_{Np_str}.h5'
        
        var_names = ['A_g_Processed', 'A_c_Processed']
        data_dict = make_data_dict(in_file,var_names,read_type,start_job,end_job)
        A_g,A_c = (data_dict[k] for k in var_names)
        # check_memory(globals())
    return A_g,A_c

In [ ]:
def GetAPrior(PROCESSING,start_job): #FIX FOR JOB_ARRAY (SO FIRST TIMESTEP WILL NOT BE BLANK)
    if start_job!=0:
        A_g_Prior,A_c_Prior=GetAData(PROCESSING,start_job-1,start_job);
        return A_g_Prior,A_c_Prior
    else:
        return 0,0
def GetAPost(PROCESSING,end_job): #*#*
    if end_job!=len(data1['time']):
        [A_g_Post,A_c_Post]=GetAData(PROCESSING,end_job-1,end_job+1)
        return A_g_Post,A_c_Post
    else:
        return 0,0
        
def SubtractA(A,A_Prior):
    D = np.zeros_like(A,dtype=np.int8)
    D[1:, :] = A[1:, :]*1 - A[:-1, :]*1
    if start_job!=0:
        D[0, :] = A[0, :]*1-A_Prior*1 #FIX FOR FIRST TIMESTEP
    return D
def AddAs(A1,A2,A1_Prior):
    S = np.zeros_like(A2,dtype=np.int8)
    S[1:, :] = A2[1:, :]*1 + A1[:-1, :]*1
    if start_job!=0:
        S[0, :] = A2[0, :]*1 + A1_Prior*1 #FIX FOR FIRST TIMESTEP
    return S

In [ ]:
def FixDetrainment(A_Post): #*#*
    #when using slurm_job_array, last timestep is not calculated
    #since data is outside of access scope
    #THIS FUNCTION FIXES THAT
    D3 = np.zeros_like(A_Post.copy(),dtype=np.int8)
    D3[1:, :] = A_Post[1:, :]*1 - A_Post[:-1, :]*1
    D3[D3 > 0] = 0
    return D3[1:]
def Apply_FixDetrainment(D, A_Post):
    D2 = np.zeros_like(D.copy())
    D2[:-1] = D[1:]  # shift D back in time
    D = D2.copy()
    if not isinstance(A_Post, int):
        D3 = FixDetrainment(A_Post)
        D[-1] = D3.copy()
    return D

def FixDetrainment_S(A1_Post,A2_Post): #*#*
    #when using slurm_job_array, last timestep is not calculated
    #since data is outside of access scope
    #THIS FUNCTION FIXES THAT
    S3 = np.zeros_like(A2_Post.copy(),dtype=np.int8)
    S3[1:, :] = A2_Post[1:, :]*1 + A1_Post[:-1, :]*1
    return S3[1:]
def Apply_FixDetrainment_S(S, A1_Post,A2_Post):
    S2 = np.zeros_like(S.copy())
    S2[:-1] = S[1:]  # shift D back in time
    S = S2.copy()
    if not isinstance(A1_Post, int) and not isinstance(A2_Post, int):
        S3 = FixDetrainment_S(A1_Post,A2_Post)
        S[-1] = S3.copy()
    return S

In [ ]:
###########################################################################################################################################################################

In [ ]:
####################################
#Functions

In [ ]:
def ed3d(data, T,Z,Y,X, A1,A2, A1_Prior,A2_Prior,A1_Post,A2_Post, type1): #*#*
    # start_time = time.time()
    """
    Function to compute 3D entrainment and update result array based on provided inputs.
    
    Returns a 3D (t,z) array containing the sum of the D array representing entrained parcels, by 1, and detrained parcels, by -1.
    The finally array is then ordered by the appropiate index using the np.add.at function
    
    Parameters:
    - A: The (t,p) lagrangian binary array.
    - T: The (t,p) lagrangian time index array.
    - Z: The (t,p) Lagrangian z index array.
    - Y: The (t,p) Lagrangian y index array.
    - X: The (t,p) Lagrangian x index array.

    """
    # Compute the difference between neighboring elements along the first axis
    D = SubtractA(A2,A2_Prior) #Entrainment/Detrainment
    #General <==> Cloudy Updraft-Transfer Entrainment
    if type1=='e':
        S_init = AddAs(A1,A2,A1_Prior) 
    elif type1=='d':
        S_init = AddAs(A2,A1,A2_Prior)
        
    # Update D for entrainment/detrainment
    if type1=='e':
        D[D < 0] = 0
    elif type1=='d':
        D[D > 0] = 0
        D = Apply_FixDetrainment(D, A2_Post) #NEED TO SHIFT TO PREVIOUS TIME TO STORE D CORRECTLY 
        S_init = Apply_FixDetrainment_S(S_init, A2_Post, A1_Post) #NEED TO SHIFT TO PREVIOUS TIME TO STORE D CORRECTLY 

    #FINDING LOCATIONS WITH TRANSFER ENTRAINMENT
    S = (S_init==2)*1
    #Removing Transfer-Entrainment where D==0 (just in case)
    S[D==0]=0 
    
    # Initialize time and vertical dimension arrays
    Nt = len(data['time']); Nz = len(data1['zh']); Ny = len(data1['yh']); Nx = len(data1['xh'])
    # Initialize result array
    result1 = np.zeros((Nt, Nz, Ny, Nx))
    result2 = np.zeros((Nt, Nz, Ny, Nx))

    # Use np.add.at to accumulate values in the result array
    np.add.at(result1, (T, Z, Y, X), D)
    np.add.at(result2, (T, Z, Y, X), S)

    # end_time = time.time()
    # print(f"Execution time: {(end_time - start_time)} seconds")
    return result1,result2

In [ ]:
def MakeDictionary(**vars): #*#*
    return vars
def RunCalculation(data, A_g,A_c, T,Z,Y,X, A_g_Prior,A_c_Prior,A_g_Post,A_c_Post): #*#*  
    # Set A based on PROCESSING state
    # print('Calculating 3D Entrainment for General Updrafts')
    [profile_array_e_g, profile_array_c_to_g_E] = ed3d(data, T,Z,Y,X, A1=A_c,A2=A_g, A1_Prior=A_c_Prior,A2_Prior=A_g_Prior,A1_Post=A_c_Post,A2_Post=A_g_Post, type1='e')
    [profile_array_d_g, profile_array_g_to_c_D] = ed3d(data, T,Z,Y,X, A1=A_c,A2=A_g, A1_Prior=A_c_Prior,A2_Prior=A_g_Prior,A1_Post=A_c_Post,A2_Post=A_g_Post, type1='d')
    
    # Set A for the second block
    # print('Calculating 3D Entrainment for Cloudy Updrafts')
    [profile_array_e_c, profile_array_g_to_c_E] = ed3d(data, T,Z,Y,X, A1=A_g,A2=A_c, A1_Prior=A_g_Prior,A2_Prior=A_c_Prior,A1_Post=A_g_Post,A2_Post=A_c_Post, type1='e')
    [profile_array_d_c, profile_array_c_to_g_D] = ed3d(data, T,Z,Y,X, A1=A_g,A2=A_c, A1_Prior=A_g_Prior,A2_Prior=A_c_Prior,A1_Post=A_g_Post,A2_Post=A_c_Post, type1='d')

    VARs=MakeDictionary(profile_array_e_g=profile_array_e_g,profile_array_e_c=profile_array_e_c,
                        profile_array_d_g=profile_array_d_g,profile_array_d_c=profile_array_d_c,
                        profile_array_g_to_c_E=profile_array_g_to_c_E,profile_array_g_to_c_D=profile_array_g_to_c_D,
                        profile_array_c_to_g_E=profile_array_c_to_g_E,profile_array_c_to_g_D=profile_array_c_to_g_D)
    return VARs

In [ ]:
def SaveProfile(VARs,job_id,PROCESSING):
    dir2=dir+'Project_Algorithms/Entrainment/SBATCH/job_out_3/'
    
    #SAVING
    # print('SAVING')
    if PROCESSING==False:
        out_file=dir2+f'3D_entrainmentdetrainment_profiles_{res}_{t_res}_{Np_str}'
    elif PROCESSING==True:
        out_file=dir2+f'3D_entrainmentdetrainment_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}'
    out_file+=f'_{job_id}.h5'
    
    vars=VARs.keys()
    t_chunk_size=end_job-start_job
    z_chunk_size=2
    InitiateArray_Job(data,out_file,vars,t_chunk_size=1,z_chunk_size=17,y_chunk_size=25,x_chunk_size=64)
    
    with h5py.File(out_file, 'a') as f: 
        for var in VARs:
            f[f'{var}'][:]=VARs[var]
    # print('done')

In [ ]:
##############################################
#RUNNING

In [ ]:
[start_slurm_job,end_slurm_job]=StartSlurmJobArray(num_jobs=num_jobs,num_slurm_jobs=num_slurm_jobs,ISRUN=True) #if ISRUN is False, then will not run using slurm_job_array

print(f"Running on Slurm_Jobs for Slurm_Job_Ids: {(start_slurm_job,end_slurm_job-1)}")

job_id_list=np.arange(start_slurm_job,end_slurm_job)
for job_id in job_id_list:
    if job_id % 1 == 0: print(f'current job_id = {job_id}')
    [start_job,end_job,index_adjust]=StartJobArray(job_id,num_jobs)

    #SLICING DATA
    [data,parcel]=GetData(data1,parcel1,start_job,end_job)
    [A_g,A_c]=GetAData(PROCESSING,start_job,end_job)
    [A_g_Prior,A_c_Prior]=GetAPrior(PROCESSING,start_job) #FIX FOR FIRST TIMESTEP
    [A_g_Post,A_c_Post]=GetAPost(PROCESSING,end_job) #FIX FOR LAST TIMESTEP #*#*#
    [T,Z,Y,X]=GetSpatialData(data,start_job,end_job)

    #RUNNING CALCULATION
    VARs=RunCalculation(data, A_g,A_c, T,Z,Y,X, A_g_Prior,A_c_Prior,A_g_Post,A_c_Post) #*#*

    #SAVING
    SaveProfile(VARs,job_id,PROCESSING)
    # check_memory(globals())
    del VARs

In [ ]:
#########################################
#RECOMBINE SEPERATE JOB_ARRAYS AFTER
recombine=False #KEEP FALSE IF JOB ARRAY IS RUNNING
# recombine=True

In [ ]:
# def Recombine(PROCESSING,num_jobs):
#     dir2=dir+'Project_Algorithms/Entrainment/SBATCH/job_out_3/'
#     dir3=dir+'Project_Algorithms/Entrainment/OUTPUT/'
    
#     if PROCESSING==False:
#         out_file=dir3+f'3D_entrainmentdetrainment_profiles_{res}_{t_res}_{Np_str}.h5'
#     elif PROCESSING==True:
#         out_file=dir3+f'3D_entrainmentdetrainment_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'
    
#     vars=["profile_array_e_g","profile_array_e_c",
#           "profile_array_d_g","profile_array_d_c",
#           "profile_array_g_to_c_E","profile_array_g_to_c_D",
#           "profile_array_c_to_g_E", "profile_array_c_to_g_D"]
#     InitiateArray(out_file,vars,t_chunk_size=50,z_chunk_size=17,y_chunk_size=25,x_chunk_size=64)
    
#     with h5py.File(out_file, 'r+') as f_out:
#         for job_id in np.arange(1,num_jobs+1):
#             if np.mod(job_id,10)==0: print(f"job_id = {job_id}")
#             [a,b,_]=StartJobArray(job_id,num_jobs)
    
#             if PROCESSING==False:
#                 in_file=dir2+f'3D_entrainmentdetrainment_profiles_{res}_{t_res}_{Np_str}_{job_id}.h5'
#             elif PROCESSING==True:
#                 in_file=dir2+f'3D_entrainmentdetrainment_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}_{job_id}.h5'
#             with h5py.File(in_file, 'r') as f_in: 
#                 for var in vars:
#                     f_out[var][a:b]=f_in[var][:]

In [ ]:
def Recombine_Dask(PROCESSING): #*#*
    #DASK-ENABLED
    import glob
    import re
    from dask.diagnostics import ProgressBar

    def recombine_func(in_files, out_file):
        # Set Dask memory management config to avoid OOM without killing the job

        matching_files = sorted(
            glob.glob(in_files),
            key=lambda f: int(re.search(r'_(\d+)\.h5$', f).group(1))
        )

        print(f'recombining {len(matching_files)} files')
        print('Forming Dask Delayed Object')
        if res=='1km' and Np_str=='1e6':
            chunks = {  # 1*95*100*256 *4/1e9 = 3.5 mb per chunk * 8 variables = ~28 mb at a time
                'phony_dim_0': 1,   # 132 chunks
                'phony_dim_1': 34,  # 1 chunk
                'phony_dim_2': 100, # 2 chunks
                'phony_dim_3': 256, # 2 chunks
            }  # Total chunks ~ 132*2*2 = 528 < 1e5 chunks
        if res=='250m':
            chunks = {  # 1*95*100*256 *4/1e9 = 9 mb per chunk * 8 variables = ~80 mb at a time
                'phony_dim_0': 1,   # 661 chunks
                'phony_dim_1': 95,  # 1 chunk
                'phony_dim_2': 100, # 8 chunks
                'phony_dim_3': 256, # 8 chunks
            }  # Total chunks ~ 661*8*8 = 42000 < 1e5 chunks

        out = xr.open_mfdataset(
            matching_files,
            engine='h5netcdf',
            concat_dim='phony_dim_0',
            combine='nested',
            phony_dims='sort',
            chunks=chunks,
            parallel=True
        )

        print('RUNNING')
        with ProgressBar():
            delayed_write = out.to_netcdf(out_file, engine='h5netcdf', compute=False)
            delayed_write.compute()

    ####################################################################################
    dir2 = dir + 'Project_Algorithms/Entrainment/SBATCH/job_out_3/'
    dir3 = dir + 'Project_Algorithms/Entrainment/OUTPUT/'

    if PROCESSING == False:
        in_files = dir2 + f'3D_entrainmentdetrainment_profiles_{res}_{t_res}_{Np_str}_*.h5'
    elif PROCESSING == True:
        in_files = dir2 + f'3D_entrainmentdetrainment_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}_*.h5'

    if PROCESSING == False:
        out_file = dir3 + f'3D_entrainmentdetrainment_profiles_{res}_{t_res}_{Np_str}.h5'
    elif PROCESSING == True:
        out_file = dir3 + f'3D_entrainmentdetrainment_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'

    recombine_func(in_files, out_file)


In [ ]:
if recombine==True:
    # Recombine(PROCESSING,num_jobs=num_jobs)
    Recombine_Dask(PROCESSING)